### DNN ( 딥뉴얼 네트워크)
#### 은닉층을 가지지 않는 인공신경망을 가지고 , 분류를 하는 모델


In [8]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 

In [33]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

redwine = pd.read_csv("winequality-red.csv", sep=";")
redwine_x= redwine.iloc[:,:-1]
redwine_y = redwine.iloc[:,-1]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(redwine_y.to_numpy().reshape(-1,1))

y_onehot = enc.transform(redwine_y.to_numpy().reshape(-1,1))

from sklearn.model_selection import train_test_split
train_x , test_x, train_y, test_y = train_test_split(redwine_x, y_onehot,test_size = 0.3, random_state = 1)

x = tf.placeholder (tf.float32, [None, 11])
y = tf.placeholder(tf.float32, [None, 6])  # y 의 클래스가 6이다

w = tf.Variable(tf.zeros([11,6]))
b = tf.Variable(tf.zeros([6]))
h = tf.nn.softmax(tf.matmul(x,w)+b)  # 출력 예측한 값

cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices = [1])

loss = tf.reduce_mean(cross_entropy)

training = tf.train.GradientDescentOptimizer(0.001).minimize(loss)  # 손실을 최소화하는 방향으로 학습해주세요.
# graident 대신에 아담 옵티마이저를 넣을 수 있다. 


# 위에 있는애들은 operation 


init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(100001):
    _, loss_value = sess.run([training, loss], feed_dict= {x : train_x, y : train_y.toarray()})
    
    # loss 는 출력을 해보려고 써넣음. [] 안에 있는 것을 두개의 값으로 반환하겠다는 뜻이며,
    # _ 는 그냥 그 값을 버리겠다는 뜻이다. 
    
    if i % 10000 == 0:
        print("step %d, %.5f" % (i, loss_value))
        
        # 만번에 한번씩, i 와 손실을 출력하기. 손실은 작을 수록 좋음 

step 0, 1.79176
step 10000, 1.08176
step 20000, 1.05125
step 30000, 1.03211
step 40000, 1.01921
step 50000, 1.01002
step 60000, 1.00318
step 70000, 0.99788
step 80000, 0.99364
step 90000, 0.99014
step 100000, 0.98717


In [34]:
pred = sess.run(h, feed_dict= {x:test_x})  # 최종적으로h 를 계산하는 것은. 최종적인 값이다.
pd.crosstab(np.argmax(test_y.toarray(), axis = 1)+3,
           np.argmax(pred, axis=1)+3)


# y 가 3등급부터 8등급까지 있는데, 원핫인코딩하면 이 숫자의 의미가 사라지고, 6개열을 가지는 2차원 배열을 만들어짐. 
# 그중에서 몇등급을 알고싶으면 인덱스를 뽑아내는데, 그 인덱스를 뽑아내는 것이 argmax 임. 
# 인덱스는 0 부터 나오지만, 등급은 3 등급부터 있기때문에 이 자료에서는 이 등급에서 +3을 해줘야한다. 
# 즉 , 사실은 3,4,5,6,7,8,등급이다 라는 뜻이당. 

col_0,5,6,7
row_0,,,
3,2,0,0
4,12,9,0
5,151,54,2
6,68,118,9
7,4,40,8
8,0,1,2


In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(test_y.toarray(), axis = 1),
              np.argmax(pred, axis =1))

0.5770833333333333

### 은닉층 

In [23]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

redwine = pd.read_csv("winequality-red.csv", sep=";")
redwine_x= redwine.iloc[:,:-1]
redwine_y = redwine.iloc[:,-1]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(redwine_y.to_numpy().reshape(-1,1))

y_onehot = enc.transform(redwine_y.to_numpy().reshape(-1,1))

from sklearn.model_selection import train_test_split
train_x , test_x, train_y, test_y = train_test_split(redwine_x, y_onehot,test_size = 0.3, random_state = 1)

x = tf.placeholder (tf.float32, [None, 11])
y = tf.placeholder(tf.float32, [None, 6]) 


w1 = tf.Variable(tf.zeros([11,20]))
b1 = tf.Variable(tf.zeros([20]))
h1 = tf.nn.relu(tf.matmul(x,w1)+b1)  # relu 말고 다른 활성화 변수를 넣어줘도 가능 

w2 = tf.Variable(tf.zeros([20,6]))
b2 = tf.Variable(tf.zeros([6]))
h = tf.nn.softmax(tf.matmul(h1,w2)+b2)  # 출력 예측한 값

cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices = [1])

loss = tf.reduce_mean(cross_entropy)

training = tf.train.GradientDescentOptimizer(0.001).minimize(loss)  # 손실을 최소화하는 방향으로 학습해주세요.

# 위에 있는애들은 operation 


init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(100001):
    _, loss_value = sess.run([training, loss], feed_dict= {x : train_x, y : train_y.toarray()})
    
    # loss 는 출력을 해보려고 써넣음. [] 안에 있는 것을 두개의 값으로 반환하겠다는 뜻이며,
    # _ 는 그냥 그 값을 버리겠다는 뜻이다. 
    
    if i % 10000 == 0:
        print("step %d, %.5f" % (i, loss_value))
        
        # 만번에 한번씩, i 와 손실을 출력하기. 손실은 작을 수록 좋음 

step 0, 1.79176
step 10000, 1.25482
step 20000, 1.21419
step 30000, 1.20277
step 40000, 1.19811
step 50000, 1.19584
step 60000, 1.19455
step 70000, 1.19367
step 80000, 1.19282
step 90000, 1.18958
step 100000, 1.17309


In [27]:
pred = sess.run(h, feed_dict= {x:test_x})  # 최종적으로h 를 계산하는 것은. 최종적인 값이다.
pd.crosstab(np.argmax(test_y.toarray(), axis = 1),
           np.argmax(pred, axis=1))


col_0,2
row_0,
0,2
1,21
2,207
3,195
4,52
5,3


In [28]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(test_y.toarray(), axis = 1),
              np.argmax(pred, axis =1))

0.43125

### 히든레이어 하나 더 넣기

In [39]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd

redwine = pd.read_csv("winequality-red.csv", sep=";")
redwine_x= redwine.iloc[:,:-1]
redwine_y = redwine.iloc[:,-1]

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(redwine_y.to_numpy().reshape(-1,1))

y_onehot = enc.transform(redwine_y.to_numpy().reshape(-1,1))

from sklearn.model_selection import train_test_split
train_x , test_x, train_y, test_y = train_test_split(redwine_x, y_onehot,test_size = 0.3, random_state = 1)

x = tf.placeholder (tf.float32, [None, 11])
y = tf.placeholder(tf.float32, [None, 6]) 


w1 = tf.Variable(tf.zeros([11,20]))
b1 = tf.Variable(tf.zeros([20]))
h1 = tf.nn.relu(tf.matmul(x,w1)+b1)  # relu 말고 다른 활성화 변수를 넣어줘도 가능  sigmoid 가능 

w2 = tf.Variable(tf.zeros([20,15]))
b2 = tf.Variable(tf.zeros([15]))
h2 = tf.nn.sigmoid(tf.matmul(h1,w2)+b2)  # 출력 예측한 값

w3 = tf.Variable(tf.zeros([15,6]))
b3 = tf.Variable(tf.zeros([6]))
h = tf.nn.softmax(tf.matmul(h2,w3)+b3)  # 출력 예측한 값

cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices = [1])

loss = tf.reduce_mean(cross_entropy)

training = tf.train.GradientDescentOptimizer(0.001).minimize(loss)  # 손실을 최소화하는 방향으로 학습해주세요.

# 위에 있는애들은 operation 


init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(100001):
    _, loss_value = sess.run([training, loss], feed_dict= {x : train_x, y : train_y.toarray()})
    
    # loss 는 출력을 해보려고 써넣음. [] 안에 있는 것을 두개의 값으로 반환하겠다는 뜻이며,
    # _ 는 그냥 그 값을 버리겠다는 뜻이다. 
    
    if i % 10000 == 0:
        print("step %d, %.5f" % (i, loss_value))
        
        # 만번에 한번씩, i 와 손실을 출력하기. 손실은 작을 수록 좋음 

step 0, 1.79176
step 10000, 1.19649
step 20000, 1.19297
step 30000, 1.19237
step 40000, 1.19221
step 50000, 1.19215
step 60000, 1.19213
step 70000, 1.19212
step 80000, 1.19212
step 90000, 1.19212
step 100000, 1.19212


In [37]:
pred = sess.run(h, feed_dict= {x:test_x})  # 최종적으로h 를 계산하는 것은. 최종적인 값이다.
pd.crosstab(np.argmax(test_y.toarray(), axis = 1),
           np.argmax(pred, axis=1))


col_0,2
row_0,
0,2
1,21
2,207
3,195
4,52
5,3


In [38]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(test_y.toarray(), axis = 1),
              np.argmax(pred, axis =1))

0.43125